In [2]:
# connecting Colab to Google Drive

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

#import pathlib
#path = pathlib.Path("/content/gdrive/My Drive/Colab Notebooks/EXAMPLE/")
#type(path)
path = "/content/gdrive/My Drive/Colab Notebooks/Datasets"

import os
os.chdir(path)
os.getcwd()
os.listdir(path)

Mounted at /content/gdrive


['.DS_Store', 'test', 'train', 'model_1', 'More Research.gdoc']

https://mc.ai/using-google-colab-for-convolutional-neural-network-with-images/

https://colab.research.google.com/drive/1a7V9Hc7ks0xxDbfCZl2_96DUGlCLuh00#scrollTo=T2GoIQXPdSkG

In [3]:
!pip install -q keras
import keras
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import pathlib

train_path = (path+"/train/")
test_path = (path+"/test/")

image_count = len(list(pathlib.Path(train_path).glob('*/*.jpg')))
print("train has "+ str(image_count) + " images.")

image_count = len(list(pathlib.Path(test_path).glob('*/*.jpg')))
print("test has "+ str(image_count) + " images.")



train has 3019 images.
test has 667 images.


https://www.youtube.com/watch?v=Gvwuyx_F-28

In [4]:
# use ImageDataGenerator to preprocess the data
from keras.preprocessing.image import ImageDataGenerator

# augment the data that we have

size = 30

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2)
test_datagen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2)

# prepare training data 
training_data = train_datagen.flow_from_directory(train_path,
                                                 #color_mode = "grayscale",
                                                 shuffle = True,
                                                 seed = 42,
                                                 target_size = (size, size),
                                                 batch_size = 10,
                                                 class_mode = 'binary',)

# prepare test data
test_data = test_datagen.flow_from_directory(test_path,
                                            #color_mode = "grayscale",
                                            shuffle = True,
                                            seed = 42,
                                            target_size = (size, size),
                                            batch_size = 6,
                                            class_mode = 'binary')


# to improve the model accuracy you can increase the number of steps_per_epoch

Found 3019 images belonging to 3 classes.
Found 667 images belonging to 3 classes.


In [ ]:
print(type(training_data))
#__getitem__(idx)
from __future__ import print_function
import numpy as np

def valid_imshow_data(data):
    data = np.asarray(data)
    if data.ndim == 2:
        return True
    elif data.ndim == 3:
        if 3 <= data.shape[2] <= 4:
            return True
        else:
            print('The "data" has 3 dimensions but the last dimension '
                  'must have a length of 3 (RGB) or 4 (RGBA), not "{}".'
                  ''.format(data.shape[2]))
            return False
    else:
        print('To visualize an image the data must be 2 dimensional or '
              '3 dimensional, not "{}".'
              ''.format(data.ndim))
        return False

x,y = training_data.next()
for i in range(0,5):
    valid_imshow_data(image)
    image = x[i]
    plt.imshow(image)
    plt.show()

#https://stackoverflow.com/questions/41695844/keras-showing-images-from-data-generator


In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten # !!!
from keras.layers import Conv2D, MaxPooling2D # !!!
from keras.optimizers import SGD

#idea: train 3(?) seperate neural networks and use them to check one another (majority wins)

no_classes=3

def model_1():
  model = Sequential()
  model.add(Conv2D(32, (3, 3), input_shape=(size, size, 3), activation="relu")) #change input shape if grayscale
  model.add(MaxPooling2D(pool_size = (2, 2)))
  model.add(Flatten())
  model.add(Dense(activation="relu", units=16))
  model.add(Dropout(.2))
  model.add(Dense(activation="sigmoid", units=16))
  model.add(Dense(units=no_classes, activation='softmax'))
  model.compile(optimizer = keras.optimizers.Adam(lr=0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
  return model

def model_2():
  model = Sequential()
  model.add(Conv2D(32, (3, 3), input_shape=(size, size, 3), activation="relu")) #change input shape if grayscale
  model.add(MaxPooling2D(pool_size = (2, 2)))
  model.add(Flatten())
  model.add(Dropout(.2))
  model.add(Dense(units=no_classes, activation='softmax'))
  model.compile(optimizer = keras.optimizers.Adam(lr=0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
  return model

def model_3():

  # https://towardsdatascience.com/covolutional-neural-network-cb0883dd6529

  model = Sequential()
    # The first two layers with 32 filters of window size 3x3
  model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(size,size,3))) # conv2d layer with relu activation
  model.add(Conv2D(32, (3, 3), activation='relu'))# application of weights (ie filter/kernel) and biases as well as a final squishification function
  model.add(MaxPooling2D(pool_size=(2, 2)))# reduces dimensions
  model.add(Dropout(0.25))# prevents overfitting by setting 25% of neurons to 0

  model.add(Conv2D(64, (3, 3), padding='same', activation='relu')) # conv2d layer with relu activation
  model.add(Conv2D(64, (3, 3), activation='relu'))# application of weights (ie filter/kernel) and biases as well as a final squishification function
  model.add(MaxPooling2D(pool_size=(2, 2)))# reduces dimensions
  model.add(Dropout(0.25))# prevents overfitting by setting 25% of neurons to 0

  model.add(Conv2D(64, (3, 3), padding='same', activation='relu')) # conv2d layer with relu activation
  model.add(Conv2D(64, (3, 3), activation='relu'))# application of weights (ie filter/kernel) and biases as well as a final squishification function
  model.add(MaxPooling2D(pool_size=(2, 2)))# reduces dimensions | take the max value over a 2x2 pooling window
  model.add(Dropout(0.25))# prevents overfitting by setting 25% of neurons to 0

  model.add(Flatten())# converting the data into a 1-dimensional array
  model.add(Dense(512, activation='relu'))# deeply connected layer
  model.add(Dropout(0.5))# prevents overfitting by setting 50% of neurons to 0
  model.add(Dense(no_classes, activation='softmax')) # deeply connected layer
  model.compile(optimizer = keras.optimizers.Adam(lr=0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
          # compiling with defined optimizer and loss function (determine learning)
  
  #model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
  return model

model = model_3()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0

In [6]:
# optimizers: https://www.dlology.com/blog/quick-notes-on-how-to-choose-optimizer-in-keras/
# 3019 (10), 667 (6)

train_count = len(list(pathlib.Path(train_path).glob('*/*.jpg'))) # 3019
test_count = len(list(pathlib.Path(test_path).glob('*/*.jpg'))) # 667
# calculate ideal steps, epochs, and validation_steps

#first train
history = model.fit(training_data,
                         steps_per_epoch = 60, # 37
                         epochs = 50, #10
                         verbose = 1,
                         validation_data = test_data, 
                         validation_steps = 12) # 21

Epoch 1/50
60/60 [==============================] - 139s 2s/step - loss: 1.1045 - accuracy: 0.3860 - val_loss: 1.0922 - val_accuracy: 0.5139
Epoch 2/50
60/60 [==============================] - 102s 2s/step - loss: 1.0098 - accuracy: 0.4316 - val_loss: 0.7511 - val_accuracy: 0.5556
Epoch 3/50
60/60 [==============================] - 85s 1s/step - loss: 0.6700 - accuracy: 0.7156 - val_loss: 0.8451 - val_accuracy: 0.6389
Epoch 4/50
60/60 [==============================] - 72s 1s/step - loss: 0.6165 - accuracy: 0.7637 - val_loss: 0.4899 - val_accuracy: 0.8472
Epoch 5/50
60/60 [==============================] - 52s 876ms/step - loss: 0.4436 - accuracy: 0.8411 - val_loss: 0.3632 - val_accuracy: 0.8750
Epoch 6/50
60/60 [==============================] - 40s 681ms/step - loss: 0.3131 - accuracy: 0.8992 - val_loss: 0.2867 - val_accuracy: 0.8889
Epoch 7/50
60/60 [==============================] - 43s 714ms/step - loss: 0.3944 - accuracy: 0.8898 - val_loss: 0.3586 - val_accuracy: 0.8472
Epoch 8/5

In [1]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

#test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

NameError: ignored

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# second train
history = model.fit(training_data,
                         steps_per_epoch = 301,
                         epochs = 10,
                         verbose = 1,
                         validation_data = test_data, 
                         validation_steps = 66)

In [ ]:
'''

saved models:

model_1 | current model3, trained on 50 epochs with the last two yeilding 100% accuracy, choosing between search, bookmark, and calendar | 3/9/21

'''

In [ ]:
loc = "model_1" #loc2 = "/Users/pattyl21/Desktop" location = "/content/gdrive/My Drive/12th Grade 2020-21/Idea #2 (technology adapter):/Patty-William (Liam), STEM/model_1.0"

recon = keras.models.load_model(loc)

recon.summary()

In [ ]:

(image_batch_train, label_batch_train) = next(iter(training_data))
print(label_batch_train)
print(type (label_batch_train))

import numpy as np
from keras.preprocessing import image
def predict_ten(ibt, mdl):
  prediction = mdl.predict(ibt[:10])
  results = [1,2,3,4,5,6,7,8,9,10]
  a=0; correct = 0
  for i in prediction:
    gpos = -1; inc = 0; saved_inc=0
    for s in i:
      if (s > gpos):
        gpos = s
        saved_inc=inc
      inc+=1
    results[a] = saved_inc
    a+=1
  return results


print(predict_ten(image_batch_train,recon))

In [ ]:
i=0; correct=0; total=0
classes = ["bookmark", "calendar", "search"]
while (i<10):
  j=0; (image_batch_train, label_batch_train) = next(iter(training_data))
  while j < len(image_batch_train):
    if (predict_ten(image_batch_train, recon)[j] == label_batch_train[j]):
      correct+=1
    else:
      plt.imshow(image_batch_train[j])
      plt.show()
      print("The CNN incorrectly recognized this image as: " + classes[int(predict_ten(image_batch_train, recon)[j])] + ", while it was actually "+ classes[int(label_batch_train[j])])
    total+=1; j+=1
  i+=1
print("Accuracy: "+ str(correct) + "/" + str(total))


Accuracy: 100/100


In [ ]:
n = 0
for i in image_batch_train[:10]:
  print("\n")
  print(label_batch_train[n])
  print(".")
  plt.imshow(i)
  plt.show()
  n+=1

In [ ]:
# Saving the model

model.save("model_")

INFO:tensorflow:Assets written to: model_1/assets
